In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load
import os
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory


# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Data Preparation

Our data directory is in a form that keras image generator's flow.from.directory method. So I will use that for creating image data for my model. Generally, it is better to augment(zoom, twist, reflect, etc.) the data in ImageDataGenerator() to help the model to generalize better. However, since there are so few data whatever options I tried the model started to underfit. So I won't touch the data.

In [2]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Paths to train and test directories
train_dir = '/kaggle/input/math482-2024-2025-1-hw-05/train/train'
test_dir = '/kaggle/input/math482-2024-2025-1-hw-05/test/test'

# Create a DataFrame for the test dataset
test_files = [f for f in os.listdir(test_dir) if f.endswith(('.png', '.jpg', '.jpeg'))]
test_df = pd.DataFrame({'filename': test_files})

# Data generators
datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2  # Reserve 20% for validation

)



test_datagen = ImageDataGenerator(rescale=1./255)  # Normalize test images

# Train data loader
train_data = datagen.flow_from_directory(
    train_dir,
    target_size=(128, 128),  # Resize images
    batch_size=16,
    class_mode='categorical', # Classes are categorical
    subset='training',
)

val_data = datagen.flow_from_directory(
    train_dir,
    target_size=(128, 128),  # Resize images
    batch_size=16,
    class_mode='categorical', # Classes are categorical
    subset='validation',
)

# Test data loader (from DataFrame)
test_df['filepath'] = test_df['filename'].apply(lambda x: os.path.join(test_dir, x))

test_data = test_datagen.flow_from_dataframe(
    test_df,
    x_col='filepath',  # Column with file paths
    y_col=None,        # Test data has no labels
    target_size=(128, 128),  # Resize images
    batch_size=16,
    class_mode=None,   # No labels for test data
    shuffle=False      # Ensure consistent order for predictions
)


Found 1280 images belonging to 6 classes.
Found 320 images belonging to 6 classes.
Found 400 validated image filenames.


# Model Building(CNN)
I am creating a CNN with 3 convolution layer. Normally, this much of a convolution layer causes the model to overfit. But, I implemented two dropout layers which eliminates %50 of the neurons and L2 regularization to adjust the weights in a way that prevents overfitting.  

In [3]:
from tensorflow.keras import datasets,layers,models,regularizers

from tensorflow.keras import Input

model = models.Sequential([
    Input(shape=(128, 128, 3)),
    layers.Conv2D(filters=32, kernel_size=(3, 3), activation='relu', kernel_regularizer=regularizers.l2(0.001)),
    layers.MaxPooling2D(pool_size=(2, 2)),


    layers.Conv2D(filters=64, kernel_size=(3, 3), activation='relu', kernel_regularizer=regularizers.l2(0.001)),
    layers.MaxPooling2D(pool_size=(2, 2)),
    
    layers.Dropout(0.5),
    
    layers.Conv2D(filters=128, kernel_size=(3, 3), activation='relu', kernel_regularizer=regularizers.l2(0.001)),
    layers.MaxPooling2D(pool_size=(2, 2)),
    
    
    
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(6, activation='softmax')
])

I will use Adam optimizer to find optimal weights classes

In [4]:
from tensorflow.keras.optimizers import Adam

model.compile(
    optimizer=Adam(learning_rate=0.001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)


# Training the Model
I shall use early stopping because I intend to train the model for high number of epochs. If the validation loss doesnt improve in 20 epoch, the model training will stop and it will revert back to the best weights. This prevents overfitting.

In [5]:
from tensorflow.keras.callbacks import EarlyStopping

early_stopping = EarlyStopping(
    monitor='val_loss',  # Monitor validation loss
    patience=20,          # Stop if no improvement for 5 epochs
    restore_best_weights=True  # Restore model to the best weights
)

history = model.fit(
    train_data,
    validation_data=val_data,  # Add validation data
    epochs=50,
    callbacks=[early_stopping]
)

Epoch 1/50


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


80/80 ━━━━━━━━━━━━━━━━━━━━ 14s 79ms/step - accuracy: 0.1674 - loss: 2.0632 - val_accuracy: 0.2000 - val_loss: 1.8753
Epoch 2/50
80/80 ━━━━━━━━━━━━━━━━━━━━ 2s 25ms/step - accuracy: 0.2001 - loss: 1.8659 - val_accuracy: 0.2000 - val_loss: 1.8492
Epoch 3/50
80/80 ━━━━━━━━━━━━━━━━━━━━ 2s 27ms/step - accuracy: 0.1874 - loss: 1.8494 - val_accuracy: 0.2000 - val_loss: 1.8354
Epoch 4/50
80/80 ━━━━━━━━━━━━━━━━━━━━ 2s 26ms/step - accuracy: 0.2233 - loss: 1.8101 - val_accuracy: 0.2062 - val_loss: 1.8221
Epoch 5/50
80/80 ━━━━━━━━━━━━━━━━━━━━ 2s 24ms/step - accuracy: 0.2027 - loss: 1.8215 - val_accuracy: 0.2000 - val_loss: 1.8138
Epoch 6/50
80/80 ━━━━━━━━━━━━━━━━━━━━ 2s 28ms/step - accuracy: 0.1750 - loss: 1.8149 - val_accuracy: 0.2094 - val_loss: 1.8090
Epoch 7/50
80/80 ━━━━━━━━━━━━━━━━━━━━ 2s 26ms/step - accuracy: 0.1704 - loss: 1.8093 - val_accuracy: 0.1969 - val_loss: 1.8077
Epoch 8/50
80/80 ━━━━━━━━━━━━━━━━━━━━ 2s 26ms/step - accuracy: 0.1921 - loss: 1.7893 - val_accuracy: 0.2094 - val_loss: 1

In [6]:
# Predict on test data
predictions = model.predict(test_data)

# Convert predictions to class indices
predicted_classes = predictions.argmax(axis=1)

# Map class indices to class labels
class_labels = list(train_data.class_indices.keys())
predicted_labels = [class_labels[i] for i in predicted_classes]


25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 54ms/step


In [7]:
import pandas as pd
import numpy as np

# Assuming `predictions` contains the predicted class indices from your model
# And `test_data` is the generator used for the test set

# Extract filenames from the test generator
filenames = test_data.filenames

# Create a DataFrame for the submission
submission_df = pd.DataFrame({
    "filename": filenames,
    "class_id": predicted_classes
})

# Ensure formatting matches the required submission format
submission_df["filename"] = submission_df["filename"].str.split("/").str[-1]  # Extract only the file name
submission_df.to_csv("submission_basic.csv", index=False)

print("Submission file created: submission_basic.csv")




Submission file created: submission_basic.csv


# Model Building (Transfer Learning)
The basic CNN model I built didn't perform well. That is probabily because of the fact that there are few data to train our model. We solve this problem with using a pretrained model which trained on large datasets. So I will use ResNet50 for this purpose.

In [17]:
from tensorflow.keras.applications import ResNet50


# Load Pre-Trained ResNet50
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(128, 128, 3))

# Freeze Base Model
base_model.trainable = False

# Add Custom Layers
x = base_model.output
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dense(256, activation="relu")(x)
x = layers.Dropout(0.5)(x)
x = layers.Dense(512, activation="relu")(x)
x = layers.Dropout(0.5)(x)
output = layers.Dense(6, activation='softmax')(x)  # 6 classes

# Create Model
model2 = models.Model(inputs=base_model.input, outputs=output)

# Compile Model
model2.compile(
    optimizer=Adam(learning_rate=0.001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# Callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)



# Training the Model (Transfer Learning)

First we just train th layers we added to the base model so that when pre-trained layers are introduced adjust their weights slightly to suit our task better.

In [19]:
# Train Model
history = model2.fit(
    train_data,
    validation_data=val_data,
    epochs=30,
    callbacks=[early_stopping]
)

# Fine-Tune Base Model
base_model.trainable = True

# Recompile with a Lower Learning Rate
model2.compile(
    optimizer=Adam(learning_rate=1e-5),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# Fine-Tune Model
history_fine = model2.fit(
    train_data,
    validation_data=val_data,
    epochs=30,
    callbacks=[early_stopping]
)

print("Training complete.")

Epoch 1/30
80/80 ━━━━━━━━━━━━━━━━━━━━ 7s 80ms/step - accuracy: 0.7810 - loss: 0.5431 - val_accuracy: 0.7375 - val_loss: 0.6337
Epoch 2/30
80/80 ━━━━━━━━━━━━━━━━━━━━ 6s 77ms/step - accuracy: 0.7999 - loss: 0.5262 - val_accuracy: 0.7344 - val_loss: 0.6283
Epoch 3/30
80/80 ━━━━━━━━━━━━━━━━━━━━ 6s 77ms/step - accuracy: 0.8047 - loss: 0.4914 - val_accuracy: 0.7250 - val_loss: 0.6248
Epoch 4/30
80/80 ━━━━━━━━━━━━━━━━━━━━ 6s 75ms/step - accuracy: 0.8089 - loss: 0.4319 - val_accuracy: 0.7500 - val_loss: 0.6403
Epoch 5/30
80/80 ━━━━━━━━━━━━━━━━━━━━ 6s 78ms/step - accuracy: 0.8128 - loss: 0.4441 - val_accuracy: 0.7312 - val_loss: 0.6202
Epoch 6/30
80/80 ━━━━━━━━━━━━━━━━━━━━ 6s 76ms/step - accuracy: 0.8146 - loss: 0.4478 - val_accuracy: 0.7312 - val_loss: 0.6235
Epoch 7/30
80/80 ━━━━━━━━━━━━━━━━━━━━ 6s 78ms/step - accuracy: 0.8230 - loss: 0.4265 - val_accuracy: 0.7531 - val_loss: 0.6018
Epoch 8/30
80/80 ━━━━━━━━━━━━━━━━━━━━ 6s 77ms/step - accuracy: 0.8484 - loss: 0.3801 - val_accuracy: 0.7750 - v

In [14]:
# Predict on test data
predictions2 = model2.predict(test_data)

# Convert predictions to class indices
predicted_classes2 = predictions2.argmax(axis=1)

# Map class indices to class labels
class_labels = list(train_data.class_indices.keys())
predicted_labels = [class_labels[i] for i in predicted_classes]

25/25 ━━━━━━━━━━━━━━━━━━━━ 3s 21ms/step


In [15]:
# Extract filenames from the test generator
filenames = test_data.filenames

# Create a DataFrame for the submission
submission_df = pd.DataFrame({
    "filename": filenames,
    "class_id": predicted_classes2
})

# Ensure formatting matches the required submission format
submission_df["filename"] = submission_df["filename"].str.split("/").str[-1]  # Extract only the file name
submission_df.to_csv("submission_transfer.csv", index=False)

print("Submission file created: submission_transfer.csv")

Submission file created: submission_transfer.csv


# Conclusion

We started with building a CNN from scratch and it didn't perform well because we have few data. So the model couldn't capture specific features in the dataset. However, we overcame this prblem with using a pre-trained model as our base model and constructing our task specific layers on top of them.